### Load in Data

In [26]:
import pandas as pd
import glob

# Get a list of all cringe csv files
cringe_list = glob.glob('Dataset(.csv)/CringeData/*.csv')
cringe_list_size = len(cringe_list)

# Initalize dataframe which will hold mergedcsv files
cringeDF = pd.DataFrame()

# Go through list and give each file a videoID and label of 0
for i in range(0, cringe_list_size):
    tempDF = pd.read_csv(cringe_list[i])
    tempDF['video_id'] = str(i) + 'c'
    tempDF['label'] = 0
    
    # Merge file into cringeDF
    cringeDF= cringeDF.append(tempDF)
    
    # Reset index
    cringeDF.reset_index()

    
# Get a list of all pain csv files
pain_list = glob.glob('Dataset(.csv)/PainData/*.csv')
pain_list_size = len(pain_list)

# Initialize dataframe which will hold merged csv files
painDF = pd.DataFrame()

# Go through list and give each file a videoID and label of 1
for i in range(0, pain_list_size):
    tempDF = pd.read_csv(pain_list[i])
    tempDF['video_id'] = str(i) + 'p'
    tempDF['label'] = 1
    
    # Merge file into cringeDF
    painDF= painDF.append(tempDF)
    
    # Reset index
    painDF.reset_index()
            

### Clean Data and Select Features

In [31]:
from sklearn.preprocessing import StandardScaler

# Filter out unsuccessful and low confidence frames
cringeDF = cringeDF.loc[cringeDF['confidence'] > 0.8]
cringeDF = cringeDF.loc[cringeDF['success'] == 1]

painDF = painDF.loc[painDF['confidence'] > 0.8] 
painDF = painDF.loc[painDF['success'] == 1]

combinedDF = cringeDF.append(painDF)
combinedDF = combinedDF.reset_index()

# Only select AU intensities
feature_list = ['AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r',
                'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r',
                'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', 'AU45_r', 'video_id', 'label']

combinedDF = combinedDF.loc[:,feature_list]

# Standardize data
standardScaler = StandardScaler()
combinedDF = standardScaler.fit_transform(combinedDF)

ValueError: could not convert string to float: '0c'